In [146]:
import findspark
findspark.init()
import pyspark
sc.stop()
sc=pyspark.SparkContext()


## To stop a spark context object use sc.stop()

In [147]:
raw_hamlet=sc.textFile('hamlet.txt')
split_hamlet=raw_hamlet.map(lambda line:line.split('\t'))

In [148]:
## Testing

first_10=split_hamlet.take(10)
first_10
for i in first_10:
    print (i[0][7:])

0
8
9
10
29
30
31
74
75
131


The first value in each element (or line from the play) is a line number that identifies the line of the play the text is from. It appears in the following format:


'hamlet@0'
'hamlet@8',
'hamlet@9',
...
We don't need the hamlet@ at the beginning of these IDs for our data analysis. Let's extract just the integer part of the ID from each line, which is much more useful.

In [177]:
def separate_ids(line):
    line[0]=line[0][7:]
    return line
hamlet_with_ids=split_hamlet.map(lambda line:separate_ids(line))
hamlet_with_ids.take(4)


[['0', '', 'HAMLET'], ['8'], ['9'], ['10', '', 'DRAMATIS PERSONAE']]

3: Remove Blank Values

Next, we want to get rid of elements that don't contain any actual words (and just have an ID as the first value). These typically represent blank lines between paragraphs or sections in the play. We also want to remove any blank values ('') within elements, which don't contain any useful information for our analysis.

In [189]:
import re
def filter_no_text_lines(line):
    if len(line)>1:
        return True
def remove_blanks(line):
    for i in line:
        line[line.index(i)]=re.sub('[^A-Za-z0-9 ]',"",i)
    for j in line:
        if len(j)==0:
            line.remove(j)
    return line 
hamlet_text_only_1=hamlet_with_ids.filter(lambda line: filter_no_text_lines(line))
hamlet_text_only=hamlet_text_only_1.map(lambda line:remove_blanks(line))
hamlet_text_only.take(15)

[['0', 'HAMLET'],
 ['10', 'DRAMATIS PERSONAE'],
 ['31', 'CLAUDIUS', 'king of Denmark KING CLAUDIUS'],
 ['75', 'HAMLET', 'son to the late and nephew to the present king'],
 ['132', 'POLONIUS', 'lord chamberlain LORD POLONIUS'],
 ['177', 'HORATIO', 'friend to Hamlet'],
 ['204', 'LAERTES', 'son to Polonius'],
 ['230', 'LUCIANUS', 'nephew to the king'],
 ['261', 'VOLTIMAND'],
 ['273', ''],
 ['276', 'CORNELIUS'],
 ['288', ''],
 ['291', 'ROSENCRANTZ', '  courtiers'],
 ['317', ''],
 ['320', 'GUILDENSTERN']]

4: Remove Pipe Characters
